In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [3]:
all_mapIdx = read_all_mapIdx()

In [50]:
Lane = gpd.read_file(fr'{inpath}\GIS\Join_LaneCounty.gdb', layer='WDs_LaneCo_Complete')

In [5]:
Lane.shape

(2488, 26)

In [6]:
len(Lane.wetdet_delin_number.unique())

1646

In [7]:
Lane.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord_Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'latitude_NewCalc',
       'longitude_NewCalc', 'NewLatLongCoordSource', 'Shape_Length',
       'Shape_Area', 'geometry'],
      dtype='object')

In [8]:
def create_code(x):
    if x == 'Portion of study area in trsqq Automated mapping':
        res = 1
    elif x == 'Entire StudyArea':
        res = 2
    elif x == 'ORTaxMap-StudyArea Intersect':
        res = 0
    else:
        res = 0
    return res

In [51]:
Lane.loc[:, 'code'] = Lane.NewLatLongCoordSource.apply(lambda x: create_code(x))

In [10]:
Lane.NewLatLongCoordSource.unique()

array(['Portion of study area in trsqq Automated mapping',
       'Entire StudyArea', 'ORTaxMap-StudyArea Intersect'], dtype=object)

In [52]:
Lane.loc[:, 'county'] = 'Lane'

In [53]:
Lane.loc[:, 'trsqq'] = Lane.trsqq.apply(lambda x: x.upper())

In [13]:
gdf = Lane.dissolve('wetdet_delin_number')
gdf['wdID'] = gdf.index
gdf.reset_index(drop=True, inplace=True)

In [14]:
gdf.shape

(1646, 27)

In [15]:
code_df = Lane[['wetdet_delin_number', 'code']].drop_duplicates()

In [16]:
code_df.shape

(1742, 2)

In [17]:
wdlist = code_df[code_df.wetdet_delin_number.duplicated()].wetdet_delin_number.unique()

In [18]:
code_df.loc[code_df.wetdet_delin_number.isin(wdlist), 'code'] = 1

In [19]:
code_df = code_df[['wetdet_delin_number', 'code']].drop_duplicates()

In [20]:
code_df.shape

(1646, 2)

In [21]:
code_df.rename(columns={'wetdet_delin_number': 'wdID'}, inplace=True)

In [22]:
gdf.drop(columns=['code'], inplace=True)

In [23]:
gdf = gdf.merge(code_df, on='wdID')

In [24]:
gdf.shape

(1646, 27)

In [25]:
Lane22 = gpd.read_file(fr'{inpath}\output\final\Lane_2022.shp')

In [26]:
Lane22.shape

(19, 3)

In [27]:
Lane22.columns

Index(['code', 'wdID', 'geometry'], dtype='object')

In [28]:
setID = 'Set006'; nm_to_add = 4370

In [29]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add, raw=False)

In [30]:
colnms = [colnm for colnm in Lane.columns if colnm in wd.columns]

In [31]:
df = wd[wd.wetdet_delin_number.isin(Lane22.wdID.unique())]

In [63]:
df.shape

(20, 27)

In [84]:
sa_gdf0, sa_gdf1, sa_df2, sa_df3, sa_gdf4 = join_WD_with_SA_by_taxmap(df=df, gdf=Lane22, mapindex=all_mapIdx)

In [66]:
len(sa_gdf0.wdID.unique())

19

In [38]:
sa_gdf0.columns

Index(['wdID', 'trsqq', 'address_location_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'ORMapNum', 'record_ID',
       'parcel_id', 'site_id', 'geometry', 'lat', 'lon', 'code', 'match'],
      dtype='object')

In [67]:
sa_gdf0.match.unique()

array([0], dtype=int64)

In [54]:
Lane[Lane.trsqq== '']

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,reissuance_response_date,project_id,site_id,latitude_NewCalc,longitude_NewCalc,NewLatLongCoordSource,Shape_Length,Shape_Area,geometry,code
1168,WD1997-0301,,3300,,EUGENE,Lane,- 17S04W26,,45.59632,-122.609,...,None,29715.0,70181,44.05579,-123.133392,Entire StudyArea,2459.014942,299905.379889,"MULTIPOLYGON (((620433.897 851135.865, 620160....",2


In [40]:
Lane[Lane.wetdet_delin_number == 'WD1997-0301']

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,reissuance_response_date,project_id,site_id,latitude_NewCalc,longitude_NewCalc,NewLatLongCoordSource,Shape_Length,Shape_Area,geometry,code
1166,WD1997-0301,17S04W26,2902,,EUGENE,Lane,- 17S04W26,,44.06248,-123.138,...,None,29715.0,33057,44.05579,-123.133392,Entire StudyArea,2459.014942,299905.379889,"MULTIPOLYGON (((620433.897 851135.865, 620160....",2
1167,WD1997-0301,17S04W26,2902,,EUGENE,Lane,- 17S04W26,,44.06248,-123.138,...,None,29715.0,70180,44.05579,-123.133392,Entire StudyArea,2459.014942,299905.379889,"MULTIPOLYGON (((620433.897 851135.865, 620160....",2
1168,WD1997-0301,,3300,,EUGENE,Lane,- 17S04W26,,45.59632,-122.609,...,None,29715.0,70181,44.05579,-123.133392,Entire StudyArea,2459.014942,299905.379889,"MULTIPOLYGON (((620433.897 851135.865, 620160....",2


In [55]:
Lane.loc[Lane.trsqq == '', 'trsqq'] = '17S04W26'

In [56]:
ndf = Lane[colnms]
ndf.loc[:, 'record_ID'] = list(range(1, ndf.shape[0]+1))

In [43]:
len(ndf.wetdet_delin_number.unique())

1646

In [44]:
gdf.columns

Index(['geometry', 'trsqq', 'parcel_id', 'address_location_desc', 'city',
       'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord_Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'latitude_NewCalc',
       'longitude_NewCalc', 'NewLatLongCoordSource', 'Shape_Length',
       'Shape_Area', 'wdID', 'code'],
      dtype='object')

In [45]:
gdf = gdf[Lane22.columns]
gdf = gdf.to_crs(epsg=2992)

In [46]:
gdf.shape

(1646, 3)

In [47]:
len(ndf[ndf.wetdet_delin_number.duplicated()].wetdet_delin_number.unique())

353

In [57]:
'' in ndf.trsqq.unique()

False

In [154]:
start = time.time()
sa_gdf0L, sa_gdf1L, sa_df2L, sa_df3L, sa_gdf4L = join_WD_with_SA_by_taxmap(df=ndf, gdf=gdf, mapindex=all_mapIdx)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 10.0 minutes to complete


In [181]:
sa_gdf4L

,wdID,trsqq,address_location_desc,city,county,site_name,site_desc,latitude,longitude,DocumentName,...,project_id,ORMapNum,record_ID,parcel_id,site_id,geometry,code,lat,lon,match


In [183]:
unmatched = sa_df2L.append(sa_df3L)

In [186]:
len(unmatched.wdID.unique())

73

In [155]:
sa_gdf0L.shape

(2274, 26)

In [159]:
sa_gdfL = sa_gdf0L[(~sa_gdf0L.geometry.isnull()) & (~sa_gdf0L.wdID.isnull())]

In [160]:
sa_gdfL.shape

(2199, 26)

In [161]:
sa_gdfL.match.unique()

array([1, 0, 5, 3, 2, 4], dtype=int64)

In [162]:
len(sa_gdfL.wdID.unique())

1613

In [164]:
sa_gdf1L.shape

(136, 6)

In [165]:
len(sa_gdf1L.wdID.unique())

33

In [166]:
sa_df3L.shape

(138, 21)

In [94]:
sa_gdf1L.wdID.unique()

array(['WD1997-0082', 'WD1997-0135', 'WD1997-0479', 'WD1998-0090',
       'WD1998-0145', 'WD1998-0373', 'WD1998-0450', 'WD1999-0059',
       'WD1999-0209', 'WD2000-0102', 'WD2000-0193', 'WD2000-0573',
       'WD2000-0579', 'WD2001-0619', 'WD2002-0178', 'WD2002-0568',
       'WD2002-0628', 'WD2003-0008', 'WD2003-0402', 'WD2006-0419',
       'WD2008-0140', 'WD2009-0018', 'WD2009-0207', 'WD2011-0055',
       'WD2011-0316', 'WD2013-0181', 'WD2013-0289', 'WD2015-0015',
       'WD2015-0021', 'WD2015-0465', 'WD2017-0382', 'WD2020-0155',
       'WD2020-0484'], dtype=object)

In [95]:
sa_gdf1L.head()

,geometry,code,wdID,ORMapNum,lat,lon
0,"POLYGON ((615248.525 847586.020, 616065.593 84...",2,WD1997-0082,2017.00S04.00W34D0--T000,44.044126,-123.150142
1,"POLYGON ((615250.807 847660.962, 616288.987 84...",2,WD1997-0082,2017.00S04.00W34DA--T000,44.044490,-123.150142
2,"MULTIPOLYGON (((660880.805 856180.946, 660882....",2,WD1997-0135,2017.00S02.00W19CD--T000,44.070708,-122.977732
3,"MULTIPOLYGON (((662106.371 855954.697, 662104....",2,WD1997-0135,2017.00S02.00W19DC--T000,44.070339,-122.974623
4,"POLYGON ((624816.061 863372.407, 624797.926 86...",2,WD1997-0479,2017.00S04.00W13AD--T000,44.089032,-123.117447


In [115]:
wid = 'WD2015-0465'

In [116]:
testgdf = sa_gdf1L[sa_gdf1L.wdID == wid]
testdf = sa_df2L[sa_df2L.wdID == wid]

In [117]:
testgdf

,geometry,code,wdID,ORMapNum,lat,lon
123,"MULTIPOLYGON (((587679.760 798757.709, 587679....",2,WD2015-0465,2019.00S05.00W1400--T000,43.909190,-123.250538
124,"MULTIPOLYGON (((588369.253 797332.500, 588334....",2,WD2015-0465,2019.00S05.00W2300--T000,43.906314,-123.251330
125,"MULTIPOLYGON (((590299.393 793296.522, 590333....",2,WD2015-0465,2019.00S05.00W2400--T000,43.898824,-123.239700
126,"MULTIPOLYGON (((591884.240 789771.369, 591870....",2,WD2015-0465,2019.00S05.00W2500--T000,43.888667,-123.237328
127,"POLYGON ((590868.995 783571.103, 590913.630 78...",2,WD2015-0465,2019.00S05.00W3600--T000,43.873186,-123.233205
128,"POLYGON ((590870.776 781948.503, 590787.275 77...",2,WD2015-0465,2020.00S05.00W0100--T000,43.859471,-123.236599
129,"MULTIPOLYGON (((590820.943 776606.056, 590819....",2,WD2015-0465,2020.00S05.00W1200--T000,43.845400,-123.235700


In [118]:
testdf

,wdID,trsqq,address_location_desc,city,county,site_name,site_desc,latitude,longitude,DocumentName,...,is_batch_file,status_name,received_date,response_date,reissuance_response_date,project_id,ORMapNum,record_ID,parcel_id,site_id
68,WD2015-0465,19S05W13,Territorial highway - MP 32.03 to 37.77,Lorane,Lane,PROJECT N END Territorial Highway 19S05W13,Reconstruction of 5.7 mi section from Gillespi...,43.9101,-123.2506,,...,N,Approved,2015-10-23T00:00:00+00:00,2016-05-05T00:00:00+00:00,None,NaN,2019.00S05.00W1300--T000,929,MANY,96500
69,WD2015-0465,20S05W13,Territorial highway - MP 32.03 to 37.77,Lorane,Lane,PROJECT S END Territorial Highway 19S05W13,Reconstruction of 5.7 mi section from Gillespi...,43.8372,-123.2382,,...,N,Approved,2015-10-23T00:00:00+00:00,2016-05-05T00:00:00+00:00,None,NaN,2020.00S05.00W1300--T000,930,MANY,96501


In [145]:
len(list(sa_gdf0L.wdID.unique()) + list(sa_gdf1L.wdID.unique()))

1647

In [158]:
sa_gdf4L.shape

(0, 26)

In [168]:
sa_gdfL.rename(columns = coldict, inplace = True)

In [171]:
sa_gdf0.rename(columns = coldict, inplace = True)

In [172]:
final_gdf = sa_gdfL.append(sa_gdf0)

In [175]:
final_gdf.to_file(fr'{inpath}\output\final\Lane_County_WD.shp')

In [179]:
sa_gdf1L.to_file(fr'{inpath}\output\final\Lane_County_WD_Unmatched.shp')

In [187]:
len([wid for wid in unmatched.wdID.unique() if wid not in final_gdf.wdID.unique()])

33

In [188]:
unmatched.to_csv(fr'{inpath}\output\to_review\Lane_County_WD_Unmatched.csv', index=False)